In [204]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

In [205]:
def loc_shape0(img0, img1, img2, s=520):
    # 가로모임 민글자
    l00 = int(s/2-img0.size[0])
    l01 = int(s/2-img0.size[1]/2)
    l10 = int(s/2)
    l11 = int(s/2-img1.size[1]/2)
    return (l00,l01), (l10,l11), None

def loc_shape1(img0, img1, img2, s=520):
    #세로모임 민글자
    l00 = int(520/2-img0.size[0]/2)
    l01 = int(520/2-img0.size[1])
    l10 = int(520/2-img1.size[0]/2)
    l11 = int(520/2)
    return (l00,l01), (l10,l11), None

def loc_shape2(img0, img1, img2, s=520):
    #섞임모임 민글자
    l00 = int(520/2-img0.size[0])
    l01 = int(520/2-img0.size[1])
    l10 = int(520/2-img1.size[0]/2)
    l11 = int(520/2)
    return (l00,l01), (l10,l11), None

def loc_shape3(img0, img1, img2, s=520):
    #가로모임 받침글자
    l00 = int(520/2-img0.size[0])
    l01 = int(520/2-img0.size[1])
    l10 = int(520/2)
    l11 = int(520/2-img1.size[1])
    l20 = int(520/2-img2.size[0]/2)
    l21 = int(520/2)
    return (l00,l01), (l10,l11), (l20, l21)

def loc_shape4(img0, img1, img2, s=520):
    #세로모임 받침글자
    l00 = int(520/2-img0.size[0]/2)
    l01 = int(520/2-img0.size[1]-img1.size[1]/2)
    l10 = int(520/2-img1.size[0]/2)
    l11 = int(520/2-img1.size[1]/2)
    l20 = int(520/2-img2.size[0]/2)
    l21 = int(520/2+img1.size[1]/2)
    return (l00,l01), (l10,l11), (l20, l21)

def loc_shape5(img0, img1, img2, s=520):
    #섞임모임 받침글자
    l00 = int(520/2-img0.size[0])
    l01 = int(520/2-img0.size[1]-img1.size[1]/2)
    l10 = int(520/2-img1.size[0]/2)
    l11 = int(520/2-img1.size[1]/2)
    l20 = int(520/2-img2.size[0]/2)
    l21 = int(520/2+img1.size[1]/2)
    return (l00,l01), (l10,l11), (l20, l21)


In [206]:
def prepare_data(letter, img0, img1, img2):
    hlist = [0,1,2,3,4,5,6,7,20] # ㅏㅐㅑㅒㅓㅔㅕㅖㅣ
    vlist = [8,12,13,17,18] # ㅗㅛㅜㅠㅡ
    clist = [9,10,11,14,15,16,19] # ㅘㅙㅚㅝㅞㅟㅢ
    
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2
    
    if (ch2 in hlist) & (ch3 == 0):
        # 가로모임 민글자
        wh0, wh1, wh2 = loc_shape0(img0, img1, img2)
        
    elif (ch2 in vlist) & (ch3 == 0):
        # 세로모임 민글자
        wh0, wh1, wh2 = loc_shape1(img0, img1, img2)

    elif (ch2 in clist) & (ch3 == 0):
        # 섞임모임 민글자
        wh0, wh1, wh2 = loc_shape2(img0, img1, img2)

    elif (ch2 in hlist) & (ch3 != 0):
        # 가로모임 받침글자
        wh0, wh1, wh2 = loc_shape3(img0, img1, img2)

    elif (ch2 in vlist) & (ch3 != 0):
        # 세로모임 받침글자
        wh0, wh1, wh2 = loc_shape4(img0, img1, img2)

    elif (ch2 in clist) & (ch3 != 0):
        # 섞임모임 받침글자
        wh0, wh1, wh2 = loc_shape5(img0, img1, img2)

    img = Image.new("L", (520, 520), 255)
    img.paste(img0, wh0)
    img.paste(img1, wh1)
    if wh2 is not None:
        img.paste(img2, wh2)
    
    label = np.zeros([4,520,520])
    label[0,:,:] = 1
    
    npwheres = np.where(np.array(img0)!=255)
    npwheres = (npwheres[0] + wh0[1], npwheres[1] + wh0[0],)
    label[0][npwheres] = 0
    label[1][npwheres] = 1
    
    npwheres = np.where(np.array(img1)!=255)
    npwheres = (npwheres[0] + wh1[1], npwheres[1] + wh1[0],)
    label[0][npwheres] = 0
    label[2][npwheres] = 1
    
    if wh2 is not None:
        npwheres = np.where(np.array(img2)!=255)
        npwheres = (npwheres[0] + wh2[1], npwheres[1] + wh2[0],)
        label[0][npwheres] = 0
        label[3][npwheres] = 1

    return img, label


def get_imgs(fd, font, letter):
    
    ch1_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    ch2_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    ch3_list = [None, 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2

    return (
        Image.open(f"{fd}/{font}/{font}__{ch1_list[ch1]}.png"),
        Image.open(f"{fd}/{font}/{font}__{ch2_list[ch2]}.png"),
        None if ch3_list[ch3] is None else Image.open(f"{fd}/{font}/{font}__{ch3_list[ch3]}.png"),
    )


def get_all_korean():

    def nextKorLetterFrom(letter):
        lastLetterInt = 15572643
        if not letter:
            return '가'
        a = letter
        b = a.encode('utf8')
        c = int(b.hex(), 16)

        if c == lastLetterInt:
            return False

        d = hex(c + 1)
        e = bytearray.fromhex(d[2:])

        flag = True
        while flag:
            try:
                r = e.decode('utf-8')
                flag = False
            except UnicodeDecodeError:
                c = c+1
                d = hex(c)
                e = bytearray.fromhex(d[2:])
        return e.decode()

    returns = []
    flag = True
    k = ''
    while flag:
        k = nextKorLetterFrom(k)
        if k is False:
            flag = False
        else:
            returns.append(k)
    return returns

In [207]:
fd = "/home/jupyter/ai_font/data/exp0717/train_parts"
fonts = os.listdir(fd)
ak = get_all_korean()

In [208]:
savefd_assembled = "/home/jupyter/ai_font/data/exp0717/train_seg/raw_assembled"
savefd_label = "/home/jupyter/ai_font/data/exp0717/train_seg/raw_label"

In [210]:
pbar = tqdm(fonts)
pass_count = 0
for font in pbar:
    os.makedirs(f"{savefd_assembled}/{font}",exist_ok=True)
    os.makedirs(f"{savefd_label}/{font}",exist_ok=True)
    for letter in ak:
        try:
            img0, img1, img2 = get_imgs(fd, font, letter)
            img, label = prepare_data(letter, img0, img1, img2)
            img.save(f"{savefd_assembled}/{font}/{font}__{letter}.png")
            np.save(f"{savefd_label}/{font}/{font}__{letter}.npy", label) 
            pbar.set_postfix(font=font, letter=letter, pass_count=str(pass_count))
        except:
            pass_count += 1

  2%|▏         | 10/426 [32:34<22:35:26, 195.50s/it, font=온글잎명숙체, letter=퇳, pass_count=931]          


OSError: [Errno 28] No space left on device: '/home/jupyter/ai_font/data/exp0717/train_seg/raw_assembled/온글잎부흥민하체'